# Environment Setup

In [1]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -U magenta pyfluidsynth

# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

# Download Salamander piano SoundFont.
# Samples by Alexander Holm: https://archive.org/details/SalamanderGrandPianoV3
# Converted to sf2 by John Nebauer: https://sites.google.com/site/soundfonts4u
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

/bin/sh: apt-get: command not found
     |████████████████████████████████| 1.5MB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 10.2MB 25.1MB/s eta 0:00:01
^C
ERROR: Operation cancelled by user
/bin/sh: gsutil: command not found


In [ ]:
import os
from magenta.models.performance_rnn import performance_sequence_generator
# N.B. the following import gives an error, to correct it you will have to open sequence_generator_bundle.py and 
# change the following line: from magenta.protobuf import generator_pb2 to magenta.music.protobuf
 
from magenta.models.shared import sequence_generator_bundle
from magenta.music.protobuf import generator_pb2
from magenta.music.protobuf import music_pb2

import magenta.music as mm

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Constants.
BUNDLE_DIR = '/tmp/'
MODEL_NAME = 'performance_with_dynamics'
BUNDLE_NAME = MODEL_NAME + '.mag'

In [ ]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)

# Generate a sequence

In [ ]:
bundle = sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 50.0  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=10)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth,
                 sf2_path='/tmp/Yamaha-C5-Salamander-JNv5.1.sf2')